In [25]:
from utils import dataset
import tqdm

In [10]:
dataset = dataset.DataSet()

Reading dataset
Total stances: 49972
Total bodies: 1683


In [12]:
dataset.articles[158]

"Hundreds of Palestinians were evacuated from their homes Sunday morning after Israeli authorities opened a number of dams near the border, flooding the Gaza Valley in the wake of a recent severe winter storm.\n\nThe Gaza Ministry of Interior said in a statement that civil defense services and teams from the Ministry of Public Works had evacuated more than 80 families from both sides of the Gaza Valley (Wadi Gaza) after their homes flooded as water levels reached more than three meters.\n\nGaza has experienced flooding in recent days amid a major storm that saw temperatures drop and frigid rain pour down.\n\nThe storm displaced dozens and caused hardship for tens of thousands, including many of the approximately 110,000 Palestinians left homeless by Israel's assault over summer.\n\nThe suffering is compounded by the fact that Israel has maintained a complete siege over Gaza for the last eight years, severely limiting electricity and the availability of fuel for generators. It has also 

In [1]:
import sys
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
from feature_engineering import word_overlap_features
from utils.dataset import DataSet
from utils.generate_test_splits import kfold_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission

from utils.system import parse_params, check_version

In [16]:
def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    X_overlap = gen_or_load_feats(word_overlap_features, h, b, "features/overlap."+name+".npy")
    #X_refuting = gen_or_load_feats(refuting_features, h, b, "features/refuting."+name+".npy")
    #X_polarity = gen_or_load_feats(polarity_features, h, b, "features/polarity."+name+".npy")
    #X_hand = gen_or_load_feats(hand_features, h, b, "features/hand."+name+".npy")

    #X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
    X = np.c_[X_hand]
    return X,y

In [ ]:
generate_features

In [5]:
check_version()
parse_params()
%tb

usage: ipykernel_launcher.py [-h] [-c]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/u2/p2narang/.local/share/jupyter/runtime/kernel-b13aec7f-7118-4039-8ca9-481571dde807.json


SystemExit: 2

In [14]:
def normalize_word(w):
    return _wnl.lemmatize(w).lower()


def get_tokenized_lemmas(s):
    return [normalize_word(t) for t in nltk.word_tokenize(s)]


def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric

    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()


def remove_stopwords(l):
    # Removes stopwords from a list of tokens
    return [w for w in l if w not in feature_extraction.text.ENGLISH_STOP_WORDS]


def gen_or_load_feats(feat_fn, headlines, bodies, feature_file):
    if not os.path.isfile(feature_file):
        feats = feat_fn(headlines, bodies)
        np.save(feature_file, feats)

    return np.load(feature_file)


In [15]:
def word_overlap_features(headlines, bodies):
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        clean_headline = get_tokenized_lemmas(clean_headline)
        clean_body = get_tokenized_lemmas(clean_body)
        features = [
            len(set(clean_headline).intersection(clean_body)) / float(len(set(clean_headline).union(clean_body)))]
        X.append(features)
    return X

In [20]:
LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
stances = dataset.stances

h, b, y = [],[],[]

for stance in stances:
    y.append(LABELS.index(stance['Stance']))
    h.append(stance['Headline'])
    b.append(dataset.articles[stance['Body ID']])

In [27]:
from tqdm import tqdm

In [33]:
_wnl = nltk.WordNetLemmatizer()

In [82]:
import re
import nltk
from sklearn import feature_extraction

In [35]:
test = word_overlap_features(h,b)

49972it [03:51, 216.09it/s]


In [36]:
type(test)

list

In [47]:
def refuting_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        # 'refute',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_headline = get_tokenized_lemmas(clean_headline)
        features = [1 if word in clean_headline else 0 for word in _refuting_words]
        X.append(features)
    return X


In [77]:
test_refute = refuting_features(h,b)

49972it [00:14, 3505.57it/s]


In [52]:
for idx, val in enumerate(test):
    if 1 in val:
        print(val)
        print(idx)
        break

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
5


In [54]:
b[5]

"'Nasa Confirms Earth Will Experience 6 Days of Total Darkness in December' Fake News Story Goes Viral"

In [55]:
b[5]

'Thousands of people have been duped by a fake news story claiming that Nasa has forecast a total blackout of earth for six days in December.\n\nThe story, entitled "Nasa Confirms Earth Will Experience 6 Days of Total Darkness in December 2014!" originated from Huzlers.com, a website well known for publishing fake stories with sensational headlines.\n\nThe bogus report read: "Nasa has confirmed that the Earth will experience 6 days of almost complete darkness and will happen from the dates Tuesday the 16 – Monday the 22 in December. The world will remain, during these three days, without sunlight due to a solar storm, which will cause dust and space debris to become plentiful and thus, block 90% sunlight.\n\n"The head of Nasa Charles Bolden who made the announcement and asked everyone to remain calm. This will be the product of a solar storm, the largest in the last 250 years for a period of 216 hours total.\n\n"Despite the six days of darkness soon to come, officials say that the eart

In [56]:
y[5]

0

In [57]:
def polarity_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]

    def calculate_polarity(text):
        tokens = get_tokenized_lemmas(text)
        return sum([t in _refuting_words for t in tokens]) % 2
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        features = []
        features.append(calculate_polarity(clean_headline))
        features.append(calculate_polarity(clean_body))
        X.append(features)
    return np.array(X)

In [58]:
test = polarity_features(h,b)

49972it [03:58, 209.13it/s]


In [64]:
for idx, val in enumerate(test):
    if val[0] == 1 and val[1] == 1:
        print(idx)

11
16
31
53
55
60
81
94
98
128
180
207
212
336
338
408
447
480
495
498
532
557
566
587
605
608
629
636
694
696
733
773
784
799
817
840
846
850
877
891
901
912
918
953
975
987
993
998
1014
1081
1089
1110
1164
1240
1245
1268
1344
1352
1357
1534
1542
1553
1572
1606
1616
1633
1639
1655
1664
1691
1693
1740
1745
1746
1748
1760
1806
1854
1856
1873
1905
1908
1941
1947
1949
1953
1958
2007
2015
2025
2036
2048
2064
2089
2139
2146
2212
2229
2238
2251
2261
2267
2273
2279
2298
2303
2315
2319
2389
2415
2434
2436
2449
2468
2484
2565
2596
2642
2649
2665
2711
2777
2804
2831
2890
2892
2912
2921
2925
2930
2950
2969
3019
3027
3029
3032
3053
3069
3095
3100
3127
3131
3140
3150
3215
3239
3346
3425
3575
3630
3635
3658
3662
3696
3720
3736
3745
3765
3769
3774
3777
3790
3801
3814
3849
3929
3935
3945
3984
3990
3994
4004
4018
4034
4054
4058
4099
4164
4193
4198
4247
4286
4294
4326
4364
4387
4398
4402
4419
4435
4479
4500
4553
4568
4598
4632
4636
4642
4670
4687
4706
4802
4826
4870
4877
4891
4907
4908
4917
4919
4920
49

44860
44877
44896
44924
44944
44945
44980
45028
45068
45071
45085
45131
45134
45177
45253
45258
45275
45287
45299
45345
45391
45396
45468
45469
45497
45554
45599
45618
45670
45674
45692
45698
45715
45720
45729
45772
45897
45901
45943
45979
45993
46021
46084
46156
46164
46216
46259
46275
46405
46412
46414
46449
46513
46526
46538
46569
46587
46598
46604
46606
46607
46608
46625
46670
46679
46726
46733
46754
46762
46771
46775
46785
46792
46816
46820
46822
46824
46828
46829
46833
46836
46846
46917
46928
46960
46962
46976
46993
46998
47042
47075
47078
47083
47131
47184
47238
47269
47272
47293
47297
47321
47332
47351
47373
47401
47470
47526
47527
47536
47579
47582
47658
47694
47718
47761
47862
47924
47929
47937
47948
48043
48047
48048
48075
48085
48094
48109
48152
48180
48183
48188
48226
48266
48275
48283
48298
48313
48347
48354
48367
48370
48386
48388
48406
48415
48480
48511
48533
48543
48552
48608
48639
48647
48657
48677
48724
48729
48778
48785
48825
48839
48862
48928
48950
48963
48977
4898

In [68]:
test[16]

array([1, 1])

In [75]:
h[49905]

'Soon Marijuana May Lead to Ticket, Not Arrest, in New York'

In [74]:
b[49905]

'Media outlets have identified "Jihadi John" - the suspect featuring in several Islamic State of Iraq and the Levant beheading videos - as Mohammed Emwazi from London.\n\nThe Washington Post newspaper named Emwazi and said he was a Briton from a well-to-do family who grew up in West London and graduated from college with a degree in computer programming.\nThe BBC later said it too had learned that Emwazi was the suspect in question.\nPolice declined to comment on the reports.\n"We are not going to confirm the identity of anyone at this stage or give an update on the progress of this live counterterrorism investigation," said Commander Richard Walton of the Metropolitan Police\'s Counter Terrorism Command in a statement.\nIn videos released by ISIL, the masked, black-clad man brandishing a knife and speaking with an English accent appears to have carried out the beheadings of three Americans and two Britons.\nThe Washington Post said Emwazi was believed to have travelled to Syria around

In [78]:
def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def chargrams(input, n):
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def append_chargrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in chargrams(" ".join(remove_stopwords(text_headline.split())), size)]
    grams_hits = 0
    grams_early_hits = 0
    grams_first_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
        if gram in text_body[:100]:
            grams_first_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    features.append(grams_first_hits)
    return features


def append_ngrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in ngrams(text_headline, size)]
    grams_hits = 0
    grams_early_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    return features

In [107]:
def hand_features(headlines, bodies):

    def binary_co_occurence(headline, body):
        # Count how many times a token in the title
        # appears in the body text.
        bin_count = 0
        bin_count_early = 0
        for headline_token in clean(headline).split(" "):
            if headline_token in clean(body):
                bin_count += 1
            if headline_token in clean(body)[:255]:
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def binary_co_occurence_stops(headline, body):
        # Count how many times a token in the title
        # appears in the body text. Stopwords in the title
        # are ignored.
        bin_count = 0
        bin_count_early = 0
        for headline_token in remove_stopwords(clean(headline).split(" ")):
            if headline_token in clean(body):
                bin_count += 1
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def count_grams(headline, body):
        # Count how many times an n-gram of the title
        # appears in the entire body, and intro paragraph

        clean_body = clean(body)
        clean_headline = clean(headline)
        features = []
        features = append_chargrams(features, clean_headline, clean_body, 2)
        features = append_chargrams(features, clean_headline, clean_body, 8)
        features = append_chargrams(features, clean_headline, clean_body, 4)
        features = append_chargrams(features, clean_headline, clean_body, 16)
        features = append_ngrams(features, clean_headline, clean_body, 2)
        features = append_ngrams(features, clean_headline, clean_body, 3)
        features = append_ngrams(features, clean_headline, clean_body, 4)
        features = append_ngrams(features, clean_headline, clean_body, 5)
        features = append_ngrams(features, clean_headline, clean_body, 6)
        return features

    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        X.append(binary_co_occurence(headline, body)
                 + binary_co_occurence_stops(headline, body)
                 + count_grams(headline, body))

        #X.append(count_grams(headline, body))


    return X

In [90]:
test_hand = hand_features(h,b)

49972it [04:35, 181.28it/s]


In [106]:
print(test_hand[11])

[11, 10, 8, 8, 20, 7, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 5, 8, 3, 7, 2, 5, 1, 3, 0]


In [95]:
def binary_co_occurence(headline, body):
        # Count how many times a token in the title
        # appears in the body text.
        bin_count = 0
        bin_count_early = 0
        for headline_token in clean(headline).split(" "):
            if headline_token in clean(body):
                bin_count += 1
            if headline_token in clean(body)[:255]:
                bin_count_early += 1
        return [bin_count, bin_count_early]

binary_co_occurence(h[11], b[11])

[11, 10]

In [96]:
def binary_co_occurence_stops(headline, body):
        # Count how many times a token in the title
        # appears in the body text. Stopwords in the title
        # are ignored.
        bin_count = 0
        bin_count_early = 0
        for headline_token in remove_stopwords(clean(headline).split(" ")):
            if headline_token in clean(body):
                bin_count += 1
                bin_count_early += 1
        return [bin_count, bin_count_early]

binary_co_occurence_stops(h[11], b[11])

[8, 8]

In [102]:
def count_grams(headline, body):
        # Count how many times an n-gram of the title
        # appears in the entire body, and intro paragraph

        clean_body = clean(body)
        print(clean_body)
        clean_headline = clean(headline)
        print(clean_headline)
        
        features = []
        features = append_chargrams(features, clean_headline, clean_body, 2)
        features = append_chargrams(features, clean_headline, clean_body, 8)
        features = append_chargrams(features, clean_headline, clean_body, 4)
        features = append_chargrams(features, clean_headline, clean_body, 16)
        return features
#         features = append_ngrams(features, clean_headline, clean_body, 2)
#         features = append_ngrams(features, clean_headline, clean_body, 3)
#         features = append_ngrams(features, clean_headline, clean_body, 4)
#         features = append_ngrams(features, clean_headline, clean_body, 5)
#         features = append_ngrams(features, clean_headline, clean_body, 6)
         

count_grams(h[11], b[11])

an iraqi official denied that a woman detained in lebanon is a wife of abu bakr al baghdadi the leader of the islamic state group adding that she is the sister of a terror suspect being held in iraq wednesday s denial comes a day after lebanese authorities said they are holding a woman believed to be al baghdadi s wife they said she was detained for travelling with a fake id and had herself claimed that she is the reclusive extremist leader s spouse this file image made from video posted on a militant website purports to show the leader of the islamic state group abu bakr al baghdadi delivering a sermon at a mosque in iraq the statement by iraq s interior ministry spokesman saad maan ibrahim adds to the confusion surrounding the identity of the woman and child who were detained about 10 days ago in northern lebanon travelling with a fake id lebanese officials said the woman saja al dulaimi is believed to be the wife of the reclusive is leader she was held by syrian authorities and free

[20, 7, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [104]:
[' '.join(x) for x in chargrams(" ".join(remove_stopwords(h[11].split())), 2)]

['W o',
 'o m',
 'm a',
 'a n',
 'n  ',
 '  d',
 'd e',
 'e t',
 't a',
 'a i',
 'i n',
 'n e',
 'e d',
 'd  ',
 '  L',
 'L e',
 'e b',
 'b a',
 'a n',
 'n o',
 'o n',
 'n  ',
 '  a',
 'a l',
 'l -',
 '- B',
 'B a',
 'a g',
 'g h',
 'h d',
 'd a',
 'a d',
 'd i',
 "i '",
 "' s",
 's  ',
 '  w',
 'w i',
 'i f',
 'f e',
 'e ,',
 ',  ',
 '  I',
 'I r',
 'r a',
 'a q',
 'q  ',
 '  s',
 's a',
 'a y',
 'y s']

In [110]:
clean_body = clean(b[11])


In [114]:
count = 0
for gram in [' '.join(x) for x in chargrams(" ".join(remove_stopwords(h[11].split())), 2)]:
#         if gram in clean_body:
#             grams_hits += 1
#         if gram in clean_body[:255]:
#             grams_early_hits += 1
        if gram in clean_body[:100]:
            count += 1
            print(gram)

t a
d i
r a
s a


In [113]:
count

4

In [115]:
clean_body[:100]

'an iraqi official denied that a woman detained in lebanon is a wife of abu bakr al baghdadi the lead'